<a href="https://colab.research.google.com/github/hhhhhenanZ/zone2assignment/blob/main/zone2assignment_User_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# zone2assignment User Guide
### GitHub link: https://github.com/hhhhhenanZ/zone2assignment/tree/main

In [4]:
# ================================
# Install required packages
# ================================
!pip install osm2gmns==1.0.1
!pip install 'geopandas[all]'
!pip install DTALite


In [5]:
import os
import shutil

In [6]:
# ================================
# Clone GitHub repo with scripts and sample data
# ================================
!git clone https://github.com/hhhhhenanZ/zone2assignment.git
%cd zone2assignment


Cloning into 'zone2assignment'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 111 (delta 36), reused 98 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (111/111), 11.41 MiB | 7.66 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/zone2assignment


In [7]:
# ================================
# Use example data from GitHub `data/` folder
# This folder includes:
# - Zone shapefile set (e.g., .shp, .dbf, .shx, .prj)
# - OSM file (e.g., Tempe.osm)
# ================================
!ls data

# ================================
# OPTIONAL: Upload custom files from your local machine
# Uncomment this block if using your own shapefiles and OSM data
# ================================
# from google.colab import files
# print("📤 Upload your shapefile set (.shp, .shx, .dbf, .prj) and an .osm file.")
# uploaded = files.upload()

# ================================
# OPTIONAL: Use Google Drive instead of uploading manually
# Make sure your Drive folder has the required files
# ================================
# from google.colab import drive
# drive.mount('/content/drive')
# !cp "/content/drive/MyDrive/your_folder/*.shp" data/
# !cp "/content/drive/MyDrive/your_folder/*.dbf" data/
# !cp "/content/drive/MyDrive/your_folder/*.shx" data/
# !cp "/content/drive/MyDrive/your_folder/*.prj" data/
# !cp "/content/drive/MyDrive/your_folder/*.osm" data/


Census_Tract_Boundary.cpg  Census_Tract_Boundary.shp  Tempe.osm
Census_Tract_Boundary.dbf  Census_Tract_Boundary.shx
Census_Tract_Boundary.prj  Census_Tract_Boundary.xml


In [9]:
# ================================
# Step 1: Extract Zone Centroid Data
# ================================
!mkdir -p Tempe_test_results
!python Read_Zone_Data.py


Shapefile loaded successfully.
Current CRS: EPSG:2223
Reprojected CRS: EPSG:4326
Available columns: Index(['OBJECTID', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME',
       'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT',
       'INTPTLON', 'Shape__Are', 'Shape__Len', 'geometry'],
      dtype='object')
Figure(1200x800)
Centroid data saved to Tempe_test_results/zone_centroid.csv


In [14]:
# Move zone_centroid.csv into the same folder as node.csv and link.csv
%cd /content/zone2assignment
!cp /content/zone2assignment/Tempe_test_results/zone_centroid.csv ./zone_centroid.csv

/content/zone2assignment


In [15]:
# ================================
# Step 2: Extract Physical Network from OSM
# ================================
!python Read_OSM_File.py

I0527 21:30:11.420208   15186 functions.cpp:43] loading data from osm file
I0527 21:30:11.423409   15186 osmnetwork.cpp:570] no valid boundary information in the osm file
I0527 21:30:15.048512   15186 osmnetwork.cpp:601] nodes: 13226 ways: 3068 relations: 0
I0527 21:30:15.056425   15186 functions.cpp:46] start to build network
I0527 21:30:15.066040   15186 functions.cpp:48] build network done
I0527 21:30:15.066824   15186 networks.cpp:471] 66 intersections consolidated
I0527 21:30:15.068854   15186 networks.cpp:380] Node activity info generated
I0527 21:30:15.068911   15186 io.cpp:91] writing network to csv files
I0527 21:30:15.107585   15186 networks.cpp:260] releasing network memory
I0527 21:30:15.114958   15186 networks.cpp:283] network memory released


In [16]:
# ================================
# Step 6: Connect Centroids to Network
# ================================
!python Connector_Generation.py

Starting to process node data...
Finding the maximum node_id in node_taz_df...
Maximum node_id in node_taz_df: 44
Adding new_node_id to node_df...
New node_id generation completed.
Filtering rows in node_df where 'zone_id' is not null...
Filtered 135 rows with non-null zone_id.
Filtered 2603 rows with null zone_id.
Saving activity_node_df to '/content/zone2assignment/connected_network/activity_node.csv'...
File saved successfully to '/content/zone2assignment/connected_network/activity_node.csv'.
Saving common_node_df to '/content/zone2assignment/connected_network/common_node.csv'...
File saved successfully to '/content/zone2assignment/connected_network/common_node.csv'.
Starting to generate connector links...
Calculating nearest TAZ nodes for activity nodes...
Calculating nearest activity nodes for taz nodes...
ave pair length: '998.4131430714165' meters.
/content/zone2assignment/Connector_Generation.py:193: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [17]:
# ================================
# Step 4: Copy and Rename for DTALite inside "validator_workspace/test_network" folder
# ================================
# Step 4.1: Create validator_workspace/test_network and copy network files
# ================================


# Create required folders
workspace_dir = "validator_workspace"
test_net_dir = os.path.join(workspace_dir, "test_network")
os.makedirs(test_net_dir, exist_ok=True)

# Copy and rename node/link files to test_network/
shutil.copy("connected_network/node_updated.csv", os.path.join(test_net_dir, "node.csv"))
shutil.copy("connected_network/link_updated.csv", os.path.join(test_net_dir, "link.csv"))

# Download default demand.csv from GitHub into test_network
!curl -o validator_workspace/test_network/demand.csv https://raw.githubusercontent.com/hhhhhenanZ/zone2assignment/main/Tempe_case/step4_validation/test_network/demand.csv

print("✅ Default demand.csv downloaded into validator_workspace/test_network")

# ================================
# OPTIONAL: Upload your own demand.csv
# This will overwrite the default one if uploaded
# ================================
# from google.colab import files
# print("📤 Optional: Upload your own demand.csv (will replace the default one).")
# uploaded = files.upload()
# if 'demand.csv' in uploaded:
#     shutil.move("demand.csv", os.path.join(test_net_dir, "demand.csv"))
#     print("✅ Custom demand.csv uploaded and placed in validator_workspace/test_network/")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4848  100  4848    0     0  18869      0 --:--:-- --:--:-- --:--:-- 18937
✅ Default demand.csv downloaded into validator_workspace/test_network


In [18]:
# ================================
# Step 4.2: Clone or copy validation tools from GitHub
# You can customize this to pull specific files if needed
# ================================
!mkdir -p validator_workspace
%cd validator_workspace
!curl -O https://raw.githubusercontent.com/hhhhhenanZ/zone2assignment/main/Tempe_case/step4_validation/GMNS_Plus_Readiness_Validator.py
!curl -O https://raw.githubusercontent.com/hhhhhenanZ/zone2assignment/main/Tempe_case/step4_validation/Network_Validator_Main.py
!git clone https://github.com/hhhhhenanZ/zone2assignment.git temp_repo
!cp -r temp_repo/Tempe_case/step4_validation/GMNS_Tools .
!rm -rf temp_repo
%cd ..

# ================================
# OPTIONAL: Upload your own validator files instead
# ================================
# Uncomment to upload a custom GMNS_Tools folder or .py files
# from google.colab import files
# print("📤 Optional: Upload your custom validator Python files or GMNS_Tools folder (as zip).")
# uploaded = files.upload()

# You can also mount Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/MyDrive/your_folder/*.py validator_workspace/
# !cp -r /content/drive/MyDrive/your_folder/GMNS_Tools validator_workspace/

/content/zone2assignment/validator_workspace
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  197k  100  197k    0     0   772k      0 --:--:-- --:--:-- --:--:--  770k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4349  100  4349    0     0  22082      0 --:--:-- --:--:-- --:--:-- 22188
Cloning into 'temp_repo'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 111 (delta 36), reused 98 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (111/111), 11.41 MiB | 12.82 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/zone2assignment


In [19]:
# ================================
# Step 4.3: Run the network validator
# ================================
%cd validator_workspace
!python Network_Validator_Main.py

/content/zone2assignment/validator_workspace
DTALite, version 0.8.1

Subdirectories:
test_network
GMNS Network Validator
Working directory: test_network
Using node file: test_network/node.csv
Using link file: test_network/link.csv
Using demand file: test_network/demand.csv
Validation level: 7

Running validation at Readiness Level 7...

SUCCESS: Zone centroids are correctly listed in non-decreasing order at the beginning of the file

First through node identified as node_id=45

Found 44 zone centroids out of 2782 total nodes

First through node identified as node_id=45

Found 45 unique zone IDs in node file
Found 24 unique zone IDs in demand file

SUCCESS: All zones in demand file exist in node file
Copied settings.csv and NEXTA.exe to test_network
sample_settings.csv file created successfully!
sample_mode_type.csv file created successfully!
number_of_modes = 1
# of nodes= 2782, largest zone id (# of zones) = 44, First Through Node ID = 45, number of links = 4467
total_base_link_volume

In [22]:
# ================================
# Step 4.4: Review Validation Outputs
# ================================

report_path = "/content/zone2assignment/validator_workspace/test_network/validation_report.json"

print("\n📄 Validation Complete!")
print("✅ Check the structured summary at:")
print(f"   {report_path}")

# OPTIONAL: Display the content of the report directly in Colab
import json
if os.path.exists(report_path):
    with open(report_path, 'r') as file:
        report = json.load(file)
        print("\n🔍 Validation Report Summary:")
        for key, value in report.items():
            print(f"- {key}: {value}")
else:
    print("⚠️ validation_report.json not found in test_network. Please ensure the validator ran successfully.")



📄 Validation Complete!
✅ Check the structured summary at:
   /content/zone2assignment/validator_workspace/test_network/validation_report.json

🔍 Validation Report Summary:
- summary: {'total': 54, 'errors': 1, 'warnings': 18, 'success': 8, 'info': 27}
- errors: [{'message': 'settings.csv not found. Required for ODME configuration.', 'field': 'settings', 'details': {}}]
- warnings: [{'message': 'mode_type.csv file not found', 'field': 'mode_type', 'details': {}}, {'message': 'settings.csv file not found', 'field': 'settings', 'details': {}}, {'message': 'Found 1333 links with unusually high hourly capacity (>3000)', 'field': 'capacity', 'details': {'high_capacity_count': 1333, 'example_links': [1, 2, 3, 4, 5]}}, {'message': 'Found 302 links with unusually high capacity per lane (>2500)', 'field': 'capacity', 'details': {'high_cap_per_lane_count': 302, 'example_links': [1, 2, 3, 4, 5]}}, {'message': 'Link type 1 has large capacity variation (min=2300.0, max=16100.0)', 'field': 'capacity

In [ ]:
# ================================
# Step 5: Traffic Assignment (Already Performed in Step 4)
# ================================
# DTALite traffic assignment was executed in Step 4 as part of the validation process.
# This step is included for clarity or additional runs if needed.


In [25]:
# ================================
# Step 6: Accessibility Check
# ================================
print("\n🧭 Accessibility Check Complete")
print("📁 Check these key output files in validator_workspace/test_network/:")
print("- inaccessible_od.csv : OD pairs with no feasible path")
print("- origin_accessibility.csv : Accessibility from each origin")
print("- destination_accessibility.csv : Accessibility for each destination")

# Optional: Preview top 5 inaccessible OD pairs
import pandas as pd
inaccessible_path = "validator_workspace/test_network/inaccessible_od.csv"

if os.path.exists(inaccessible_path):
    df = pd.read_csv(inaccessible_path)
    print("\n🔍 Preview: Top inaccessible OD pairs:")
    display(df.head())
else:
    print("✅ All OD pairs appear to be connected. No inaccessible_od.csv found.")





🧭 Accessibility Check Complete
📁 Check these key output files in validator_workspace/test_network/:
- inaccessible_od.csv : OD pairs with no feasible path
- origin_accessibility.csv : Accessibility from each origin
- destination_accessibility.csv : Accessibility for each destination

🔍 Preview: Top inaccessible OD pairs:


,mode_type,origin_zone_id,destination_zone_id,google_maps_http_link
0,auto,1,5,"https://www.google.com/maps/dir/33.329291,-111..."
1,auto,2,5,"https://www.google.com/maps/dir/33.329354,-111..."
2,auto,3,5,"https://www.google.com/maps/dir/33.327312,-111..."
3,auto,4,5,"https://www.google.com/maps/dir/33.418651,-111..."
4,auto,6,5,"https://www.google.com/maps/dir/33.385992,-111..."


In [26]:
# ================================
# Final Step: Zip test_network folder for download
# ================================
import shutil

shutil.make_archive("test_network_results", 'zip', "validator_workspace/test_network")
print("\n📦 Zipped test_network folder created: test_network_results.zip")

# Provide download link
from google.colab import files
files.download("test_network_results.zip")


📦 Zipped test_network folder created: test_network_results.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
#Note: Clear files


# Step 0: Move to a safe location to allow cleanup
%cd /

# Step 1: Remove all files and folders inside /content
!rm -rf /content/*

# Step 2: Reset all Python variables
%reset -f

# Step 3: Clear all cell outputs
from IPython.display import clear_output
clear_output()

# Step 4: Return to /content as working directory
%cd /content

print("🔄 All files, variables, and outputs have been cleared.")

/content
🔄 All files, variables, and outputs have been cleared.
